## PLSX - Phuture Liquid Staking Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0
min_weight = 0.005
max_weight = 0.15
circ_supply_threshold = 0
liveness_threshold = 30 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["liquid-staking-tokens","liquid-restaking-tokens"]

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                100000,
                                coingecko_category)
print(max_slippage)

-0.04


### Token inclusion criteria

1. **Project and token characteristics**
    1. The token should be listed on CoinGecko with pricing data spanning at least 1 month prior to the date of inclusion in the index.
    1. No rebasing tokens, the staking yield should accrue to the token's price.
    1. The token should be 100% collateralised by the underlying assets.
   <br>
1. **Pricing Requirements**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange on a supported chain.
    1. The price should experience no more than 5% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety.
1. **Weighting Requirements**
    1. Each token is weighted according to its market cap.
    1. The maximum weight any one token can have is 15%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the maximum weight the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    ids_to_replace=[['staked-ether'],['wrapped-steth']],  
    remove_category_assets=['frax-ether','sweth'],
    values_to_update=[{'id':'benqi-liquid-staked-avax','category':'symbol','value':"sAVAX"},
                      {'id':'rocket-pool-eth','category':'symbol','value':"rETH"},
                      {'id':'wrapped-steth','category':'symbol','value':"wstETH"},
                      {'id':'staked-frax-ether','category':'symbol','value':"sfrxETH"},
                      {'id':'stader-ethx','category':'symbol','value':"ETHx"},
                      {'id':'stakewise-v3-oseth','category':'symbol','value':"osETH"},
                      {'id':'restaked-swell-eth','category':'circulating_supply','value':50789.180445528569814269},
                      {'id':'restaked-swell-eth','category':'total_supply','value':50789.180445528569814269},
                      {'id':'restaked-swell-eth','category':'market_cap','value':50789.180445528569814269*3500},
                      {'id':'restaked-swell-eth','category':'symbol','value':'rswETH'},
                      {'id':'renzo-restaked-eth','category':'circulating_supply','value':359925.079372744947154147},
                      {'id':'renzo-restaked-eth','category':'total_supply','value':359925.079372744947154147},
                      {'id':'renzo-restaked-eth','category':'market_cap','value':359925.079372744947154147*3500},
                      {'id':'renzo-restaked-eth','category':'symbol','value':'ezETH'},
                      ],
    onchain_oracles=['coinbase-wrapped-staked-eth','lido-staked-matic','sweth','kelp-dao-restaked-eth','stader-maticx','wrapped-eeth'],
    weight_by=["yield",methodology_class.get_yield_data]
    )
liquidity

/Users/olivermehr/Documents/GitHub/rebalancing-data/PLSX/../methodology.py:166: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.category_data = pd.concat(dataframe_list, axis=0)


zksync not supported
blast not supported
linea not supported
mode not supported
mantle not supported
Manually calculating slippage for staked-frax-ether on ethereum
moonbeam not supported
staked-frax-ether on fantom returned an invalid API response
Manually calculating slippage for staked-frax-ether on optimistic-ethereum
Manually calculating slippage for staked-frax-ether on arbitrum-one
staked-frax-ether on polygon-pos returned an invalid API response
staked-frax-ether on binance-smart-chain returned an invalid API response
stader-maticx on ethereum returned an invalid API response
manta-pacific not supported
ankreth on fantom returned an invalid API response
Manually calculating slippage for ankreth on optimistic-ethereum
ankreth on avalanche returned an invalid API response
blast not supported
scroll not supported
polygon-zkevm not supported
linea not supported
ankreth on arbitrum-one returned an invalid API response
mode not supported
trufin-staked-matic on ethereum returned an in

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,fantom,optimistic-ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain
id,,,,,,,,,,,,
kelp-dao-restaked-eth,NaN,NaN,NaN,NaN,-0.000817,NaN,NaN,NaN,ethereum,-0.000817,-0.000817,ethereum
dinero-staked-eth,NaN,NaN,NaN,NaN,-0.001085,NaN,NaN,NaN,ethereum,-0.001085,-0.001085,ethereum
wrapped-steth,-0.001246,NaN,NaN,NaN,-0.001092,NaN,-0.003441,-0.019691,arbitrum-one,-0.001092,-0.001246,ethereum
wrapped-eeth,-0.002044,NaN,NaN,NaN,-0.001093,NaN,NaN,NaN,arbitrum-one,-0.001093,-0.002044,ethereum
rocket-pool-eth,-0.001484,NaN,-0.002472,NaN,-0.001103,NaN,-0.004250,-0.957322,arbitrum-one,-0.001103,-0.001484,ethereum
coinbase-wrapped-staked-eth,-0.003080,NaN,-0.001957,NaN,-0.001128,NaN,-0.999275,-0.848834,arbitrum-one,-0.001128,-0.003080,ethereum
mantle-staked-ether,NaN,NaN,NaN,NaN,-0.001295,NaN,NaN,NaN,ethereum,-0.001295,-0.001295,ethereum
renzo-restaked-eth,-0.044255,NaN,-0.067405,-0.899690,-0.001322,NaN,NaN,NaN,ethereum,-0.001322,-0.001322,ethereum
stakewise-v3-oseth,NaN,NaN,NaN,NaN,-0.001344,NaN,NaN,NaN,ethereum,-0.001344,-0.001344,ethereum


In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
benqi-liquid-staked-avax,sAVAX,3.048700e+08,0.110678,7254,0x2b2c81e08f1af8835a78bb2a90ae924ace0ea4be,avalanche
stader-maticx,MATICX,1.145517e+08,0.100579,6592,0xfa68fb4628dff1028cfec22b4162fccd0d45efb6,polygon-pos
lido-staked-matic,STMATIC,0.000000e+00,0.082830,5429,0x3a58a54c066fdc0f2d55fc9c89f0415c92ebf3c4,polygon-pos
coinbase-wrapped-staked-eth,CBETH,7.001503e+08,0.078735,5160,0x1debd73e752beaf79865fd6446b0c970eae7732f,arbitrum-one
mantle-staked-ether,METH,1.782559e+09,0.078271,5130,0xd5f7838f5c461feff7fe49ea5ebaf7728bb0adfa,ethereum
restaked-swell-eth,rswETH,1.777621e+08,0.070695,4633,0xfae103dc9cf190ed75350761e95403b7b8afa6c0,ethereum
wrapped-eeth,WEETH,4.433122e+09,0.068039,4459,0x35751007a407ca6feffe80b3cb397736d2cf4dbe,arbitrum-one
renzo-restaked-eth,ezETH,1.259738e+09,0.067842,4446,0xbf5495efe5db9ce00f80364c8b423567e58d2110,ethereum
staked-frax-ether,sfrxETH,5.895473e+08,0.066023,4327,0x95ab45875cffdba1e5f451b950bc2e42c0053f39,arbitrum-one
